In [1]:
from typing import List, Dict, Any
import pandas as pd
import io

pdt = pd.to_datetime

## load data

raw ="""START,FINISH
0.000000    ,10.000000
10.000000   ,4500.182997
5000.00    ,7000.000000
6000   ,8500.687227
9850.123,9990.000000
"""

buf_bytes = io.StringIO(raw)
df = pd.read_csv(buf_bytes)

# Source: https://stackoverflow.com/questions/57882621/efficient-merge-overlapping-intervals-in-same-pandas-dataframe-with-start-and-fi
# Source: https://stackoverflow.com/a/57883722

## solution

df.sort_values("START", inplace=True)

## This line compares if START of next row is greater than FINISH of current
## row ("shift" shifts down FINISH by one row). The value of expression before
## cumsum will be True if interval breaks (i.e. cannot be merged), so  
## cumsum will increment group value when interval breaks (cum sum treats True=1, False=0)
df["group"] = (df["START"] > df["FINISH"].shift()).cumsum()

## this returns min value of "START" column from a group and max value fro m "FINISH"
result = df.groupby("group").agg({"START":"min", "FINISH": "max"})
display(result)

,START,FINISH
group,,
0,0.000,4500.182997
1,5000.000,8500.687227
2,9850.123,9990.000000


In [2]:
df

,START,FINISH,group
0,0.000,10.000000,0
1,10.000,4500.182997,0
2,5000.000,7000.000000,1
3,6000.000,8500.687227,1
4,9850.123,9990.000000,2


In [3]:
df2 = pd.DataFrame(data=[
    ["2010-01-01", "2010-01-02", "A1", "B1"],
    ["2010-01-03", "2010-01-04", "A1", "B1"],
    ["2010-01-01", "2010-01-05", "A2", "B2"],
    ["2010-01-02", "2010-01-03", "A1", "B1"],
    ["2010-01-01", "2010-01-05", "A2", "B2"],
    ["2010-01-07", "2010-01-09", "A1", "B1"],
    ["2010-01-08", "2010-01-10", "A2", "B3"],
    ["2010-01-02", "2010-01-05", "A2", "B2"],
], columns=["start", "end", "from", "to"]
)

df2

,start,end,from,to
0,2010-01-01,2010-01-02,A1,B1
1,2010-01-03,2010-01-04,A1,B1
2,2010-01-01,2010-01-05,A2,B2
3,2010-01-02,2010-01-03,A1,B1
4,2010-01-01,2010-01-05,A2,B2
5,2010-01-07,2010-01-09,A1,B1
6,2010-01-08,2010-01-10,A2,B3
7,2010-01-02,2010-01-05,A2,B2


In [4]:
pdt(df2.start)

0   2010-01-01
1   2010-01-03
2   2010-01-01
3   2010-01-02
4   2010-01-01
5   2010-01-07
6   2010-01-08
7   2010-01-02
Name: start, dtype: datetime64[ns]

In [5]:
df2.start = pdt(df2.start)
df2.end = pdt(df2.end)

df2

,start,end,from,to
0,2010-01-01,2010-01-02,A1,B1
1,2010-01-03,2010-01-04,A1,B1
2,2010-01-01,2010-01-05,A2,B2
3,2010-01-02,2010-01-03,A1,B1
4,2010-01-01,2010-01-05,A2,B2
5,2010-01-07,2010-01-09,A1,B1
6,2010-01-08,2010-01-10,A2,B3
7,2010-01-02,2010-01-05,A2,B2


In [6]:
df3 = df2.sort_values(["from", "to", "start", "end"])

df3

,start,end,from,to
0,2010-01-01,2010-01-02,A1,B1
3,2010-01-02,2010-01-03,A1,B1
1,2010-01-03,2010-01-04,A1,B1
5,2010-01-07,2010-01-09,A1,B1
2,2010-01-01,2010-01-05,A2,B2
4,2010-01-01,2010-01-05,A2,B2
7,2010-01-02,2010-01-05,A2,B2
6,2010-01-08,2010-01-10,A2,B3


In [7]:
df4 = df3.drop_duplicates()

df4

,start,end,from,to
0,2010-01-01,2010-01-02,A1,B1
3,2010-01-02,2010-01-03,A1,B1
1,2010-01-03,2010-01-04,A1,B1
5,2010-01-07,2010-01-09,A1,B1
2,2010-01-01,2010-01-05,A2,B2
7,2010-01-02,2010-01-05,A2,B2
6,2010-01-08,2010-01-10,A2,B3


In [8]:
groups = df4.groupby(["from", "to"])
groups

In [9]:
for g in groups:
    print(type(g), g)

<class 'tuple'> (('A1', 'B1'),        start        end from  to
0 2010-01-01 2010-01-02   A1  B1
3 2010-01-02 2010-01-03   A1  B1
1 2010-01-03 2010-01-04   A1  B1
5 2010-01-07 2010-01-09   A1  B1)
<class 'tuple'> (('A2', 'B2'),        start        end from  to
2 2010-01-01 2010-01-05   A2  B2
7 2010-01-02 2010-01-05   A2  B2)
<class 'tuple'> (('A2', 'B3'),        start        end from  to
6 2010-01-08 2010-01-10   A2  B3)


In [10]:
def merge_intervals_only(
    df: pd.DataFrame,
    start_col: str = "start",
    end_col: str = "end",
    agg_remaining_cols: Dict[str, Any] = dict(),
) -> pd.DataFrame:
    tmp_df = df.copy()
    ## This line compares if START of next row is greater than FINISH of current
    ## row ("shift" shifts down FINISH by one row). The value of expression before
    ## cumsum will be True if interval breaks (i.e. cannot be merged), so  
    ## cumsum will increment group value when interval breaks (cum sum treats True=1, False=0)
    tmp_df["_tmp_group"] = (tmp_df[start_col] > tmp_df[end_col].shift()).cumsum()
    ## this returns min value of "START" column from a group and max value from "FINISH"
    result = tmp_df.groupby("_tmp_group").agg({start_col: "min", end_col: "max", **agg_remaining_cols})
    result.reset_index(drop=True, inplace=True)
    return result

def merge_intervals_for_group(
    group: pd.core.groupby.DataFrameGroupBy,
    *,
    groupby_cols: List[str],
    start_col: str,
    end_col: str,
    agg_remaining_cols: Dict[str, Any] = dict(),
) -> pd.DataFrame:
    groupby_key, df = group

    result = merge_intervals_only(df, start_col=start_col, end_col=end_col, agg_remaining_cols=agg_remaining_cols)
    for col, val in zip(groupby_cols, groupby_key):
        result[col] = val

    return result

In [11]:
start_col, end_col = "start", "end"
groupby_cols = ["from", "to"]

list_dfs_merged_intervals = [
    merge_intervals_for_group(group, groupby_cols=groupby_cols, start_col=start_col, end_col=end_col)
    for group in groups
]

final_df = pd.concat(list_dfs_merged_intervals)
final_df.sort_values(["start", "end", "from", "to"], inplace=True)
final_df.reset_index(drop=True, inplace=True)

final_df

,start,end,from,to
0,2010-01-01,2010-01-04,A1,B1
1,2010-01-01,2010-01-05,A2,B2
2,2010-01-07,2010-01-09,A1,B1
3,2010-01-08,2010-01-10,A2,B3


In [12]:
def merge_intervals_based_on_cols(
    df: pd.DataFrame,
    groupby_cols: List[str],
    start_col="start",
    end_col="end",
    agg_remaining_cols: Dict[str, Any] = dict(),
) -> pd.DataFrame:
    # Step 1. Sort by groupby_cols and the start and end columns
    # Sorting by start_col, end_col is essential to handle unsorted input
    # Be careful and make a copy of input and not modifying it
    sortby_cols = [*groupby_cols, start_col, end_col]
    df = df.sort_values(sortby_cols)
    
    # Step 2. Do the groupby for non-interval columns
    groups = df.groupby(groupby_cols)

    # Step 3. Merge intervals for each group
    list_dfs_merged_intervals = [
        merge_intervals_for_group(
            group,
            groupby_cols=groupby_cols,
            start_col=start_col,
            end_col=end_col,
            agg_remaining_cols=agg_remaining_cols,
        )
        for group in groups
    ]

    # Step 4. Merge groups together, do sorting and clean the index
    final_df = pd.concat(list_dfs_merged_intervals)
    final_df.sort_values([start_col, end_col, *groupby_cols], inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df

In [13]:
merge_intervals_based_on_cols(df4, groupby_cols=groupby_cols, start_col=start_col, end_col=end_col)

,start,end,from,to
0,2010-01-01,2010-01-04,A1,B1
1,2010-01-01,2010-01-05,A2,B2
2,2010-01-07,2010-01-09,A1,B1
3,2010-01-08,2010-01-10,A2,B3


In [14]:
# It's crucial that we must sort df2 by start and end dates
merge_intervals_based_on_cols(df2, groupby_cols=groupby_cols, start_col=start_col, end_col=end_col)

,start,end,from,to
0,2010-01-01,2010-01-04,A1,B1
1,2010-01-01,2010-01-05,A2,B2
2,2010-01-07,2010-01-09,A1,B1
3,2010-01-08,2010-01-10,A2,B3


In [15]:
# Dropping duplicates from df3 is not crucial, but good to do
merge_intervals_based_on_cols(df3, groupby_cols=groupby_cols, start_col=start_col, end_col=end_col)

,start,end,from,to
0,2010-01-01,2010-01-04,A1,B1
1,2010-01-01,2010-01-05,A2,B2
2,2010-01-07,2010-01-09,A1,B1
3,2010-01-08,2010-01-10,A2,B3


In [16]:
df5 = pd.DataFrame(data=[
    ["2010-01-01", "2010-01-02", "A1", "B1", "S1", "P1", 0.1],
    ["2010-01-03", "2010-01-04", "A1", "B1", "S2", "P2", 0.2],
    ["2010-01-01", "2010-01-05", "A2", "B2", "S3", "P3", 0.3],
    ["2010-01-02", "2010-01-03", "A1", "B1", "S4", "P4", 0.4],
    ["2010-01-01", "2010-01-05", "A2", "B2", "S5", "P5", 0.5],
    ["2010-01-07", "2010-01-09", "A1", "B1", "S6", "P6", 0.6],
    ["2010-01-08", "2010-01-10", "A2", "B3", "S7", "P7", 0.7],
    ["2010-01-02", "2010-01-05", "A2", "B2", "S8", "P8", 0.8],
], columns=["start", "end", "from", "to", "source", "provider", "cost"]
)

df5

,start,end,from,to,source,provider,cost
0,2010-01-01,2010-01-02,A1,B1,S1,P1,0.1
1,2010-01-03,2010-01-04,A1,B1,S2,P2,0.2
2,2010-01-01,2010-01-05,A2,B2,S3,P3,0.3
3,2010-01-02,2010-01-03,A1,B1,S4,P4,0.4
4,2010-01-01,2010-01-05,A2,B2,S5,P5,0.5
5,2010-01-07,2010-01-09,A1,B1,S6,P6,0.6
6,2010-01-08,2010-01-10,A2,B3,S7,P7,0.7
7,2010-01-02,2010-01-05,A2,B2,S8,P8,0.8


In [17]:
# The remaining columns will be dropped if we don't process them and we don't want that!
merge_intervals_based_on_cols(df5, groupby_cols=groupby_cols, start_col=start_col, end_col=end_col)

,start,end,from,to
0,2010-01-01,2010-01-04,A1,B1
1,2010-01-01,2010-01-05,A2,B2
2,2010-01-07,2010-01-09,A1,B1
3,2010-01-08,2010-01-10,A2,B3


In [18]:
# Specify aggregation for each of the remaining columns.
# Unspecified columns will be omitted in the output.
agg_remaining_cols = {
    "source": "first",
    "provider": ", ".join,
    "cost": "sum",
}

In [19]:
groupby_agg = df5.groupby([*groupby_cols, start_col, end_col]).agg({**agg_remaining_cols}).reset_index()
#type(groupby_agg)
groupby_agg

,from,to,start,end,source,provider,cost
0,A1,B1,2010-01-01,2010-01-02,S1,P1,0.1
1,A1,B1,2010-01-02,2010-01-03,S4,P4,0.4
2,A1,B1,2010-01-03,2010-01-04,S2,P2,0.2
3,A1,B1,2010-01-07,2010-01-09,S6,P6,0.6
4,A2,B2,2010-01-01,2010-01-05,S3,"P3, P5",0.8
5,A2,B2,2010-01-02,2010-01-05,S8,P8,0.8
6,A2,B3,2010-01-08,2010-01-10,S7,P7,0.7


In [20]:
merge_intervals_based_on_cols(
    df5, groupby_cols=groupby_cols, start_col=start_col, end_col=end_col,
    agg_remaining_cols=agg_remaining_cols
)

,start,end,source,provider,cost,from,to
0,2010-01-01,2010-01-04,S1,"P1, P4, P2",0.7,A1,B1
1,2010-01-01,2010-01-05,S3,"P3, P5, P8",1.6,A2,B2
2,2010-01-07,2010-01-09,S6,P6,0.6,A1,B1
3,2010-01-08,2010-01-10,S7,P7,0.7,A2,B3


In [21]:
# Python is not OCaml :(
#from functools import partial
#from pipetools import pipe
#pd.DataFrame.pipe(df5, pd.DataFrame.sort_values)

# https://realpython.com/pandas-groupby/

In [22]:
df5.start

0    2010-01-01
1    2010-01-03
2    2010-01-01
3    2010-01-02
4    2010-01-01
5    2010-01-07
6    2010-01-08
7    2010-01-02
Name: start, dtype: object

In [23]:
# https://hub.docker.com/r/owlbarn/owl/
# https://ocaml.xyz/book/dataframe.html
# https://github.com/akabe/ocaml-jupyter
# https://ocaml.xyz/book/ndarray.html

for col in ["start", "end", "from", "to", "source", "provider", "cost"]:
    print(f"""let {col}_values = Dataframe.pack_string_series [|{'"; "'.join(map(str, df5[col].values))}|];;""")

let start_values = Dataframe.pack_string_series [|2010-01-01"; "2010-01-03"; "2010-01-01"; "2010-01-02"; "2010-01-01"; "2010-01-07"; "2010-01-08"; "2010-01-02|];;
let end_values = Dataframe.pack_string_series [|2010-01-02"; "2010-01-04"; "2010-01-05"; "2010-01-03"; "2010-01-05"; "2010-01-09"; "2010-01-10"; "2010-01-05|];;
let from_values = Dataframe.pack_string_series [|A1"; "A1"; "A2"; "A1"; "A2"; "A1"; "A2"; "A2|];;
let to_values = Dataframe.pack_string_series [|B1"; "B1"; "B2"; "B1"; "B2"; "B1"; "B3"; "B2|];;
let source_values = Dataframe.pack_string_series [|S1"; "S2"; "S3"; "S4"; "S5"; "S6"; "S7"; "S8|];;
let provider_values = Dataframe.pack_string_series [|P1"; "P2"; "P3"; "P4"; "P5"; "P6"; "P7"; "P8|];;
let cost_values = Dataframe.pack_string_series [|0.1"; "0.2"; "0.3"; "0.4"; "0.5"; "0.6"; "0.7"; "0.8|];;


In [24]:
ocaml_cmd = """
let start_values = Dataframe.pack_string_series [|"2010-01-01"; "2010-01-03"; "2010-01-01"; "2010-01-02"; "2010-01-01"; "2010-01-07"; "2010-01-08"; "2010-01-02"|];;
let end_values = Dataframe.pack_string_series [|"2010-01-02"; "2010-01-04"; "2010-01-05"; "2010-01-03"; "2010-01-05"; "2010-01-09"; "2010-01-10"; "2010-01-05"|];;
let from_values = Dataframe.pack_string_series [|"A1"; "A1"; "A2"; "A1"; "A2"; "A1"; "A2"; "A2"|];;
let to_values = Dataframe.pack_string_series [|"B1"; "B1"; "B2"; "B1"; "B2"; "B1"; "B3"; "B2"|];;
let source_values = Dataframe.pack_string_series [|"S1"; "S2"; "S3"; "S4"; "S5"; "S6"; "S7"; "S8"|];;
let provider_values = Dataframe.pack_string_series [|"P1"; "P2"; "P3"; "P4"; "P5"; "P6"; "P7"; "P8"|];;
let cost_values = Dataframe.pack_float_series [|0.1; 0.2; 0.3; 0.4; 0.5; 0.6; 0.7; 0.8|];;

let df = Dataframe.make [|"start"; "end"; "from"; "to"; "source"; "provider"; "cost"|] ~data:
[|start_values; end_values; from_values; to_values; source_values; provider_values; cost_values|];;
"""

In [25]:
# https://www.analyticsvidhya.com/blog/2021/02/is-pypolars-the-new-alternative-to-pandas/
# https://docs.rs/polars/0.14.7/polars/
import polars as pl

In [26]:
df6 = pl.DataFrame(data={col: df5[col] for col in df5.columns})

df6

start,end,from,to,source,provider,cost
str,str,str,str,str,str,f64
"""2010-01-01""","""2010-01-02""","""A1""","""B1""","""S1""","""P1""",0.1
"""2010-01-03""","""2010-01-04""","""A1""","""B1""","""S2""","""P2""",0.2
"""2010-01-01""","""2010-01-05""","""A2""","""B2""","""S3""","""P3""",0.3
"""2010-01-02""","""2010-01-03""","""A1""","""B1""","""S4""","""P4""",0.4
"""2010-01-01""","""2010-01-05""","""A2""","""B2""","""S5""","""P5""",0.5
"""2010-01-07""","""2010-01-09""","""A1""","""B1""","""S6""","""P6""",0.6
"""2010-01-08""","""2010-01-10""","""A2""","""B3""","""S7""","""P7""",0.7
"""2010-01-02""","""2010-01-05""","""A2""","""B2""","""S8""","""P8""",0.8


In [27]:
# "provider": ", ".join, 
df7 = df6.groupby(["start", "end", "from", "to"]).agg({"source": "first", "cost": "sum", "provider": "last"})

df7

start,end,from,to,source_first,provider_last,cost_sum
str,str,str,str,str,str,f64
"""2010-01-07""","""2010-01-09""","""A1""","""B1""","""S6""","""P6""",0.6
"""2010-01-02""","""2010-01-05""","""A2""","""B2""","""S8""","""P8""",0.8
"""2010-01-01""","""2010-01-02""","""A1""","""B1""","""S1""","""P1""",0.1
"""2010-01-01""","""2010-01-05""","""A2""","""B2""","""S3""","""P5""",0.8
"""2010-01-03""","""2010-01-04""","""A1""","""B1""","""S2""","""P2""",0.2
"""2010-01-08""","""2010-01-10""","""A2""","""B3""","""S7""","""P7""",0.7
"""2010-01-02""","""2010-01-03""","""A1""","""B1""","""S4""","""P4""",0.4


In [28]:
df8 = df7.sort(["from", "to", "start", "end"])

df8

start,end,from,to,source_first,provider_last,cost_sum
str,str,str,str,str,str,f64
"""2010-01-01""","""2010-01-02""","""A1""","""B1""","""S1""","""P1""",0.1
"""2010-01-02""","""2010-01-03""","""A1""","""B1""","""S4""","""P4""",0.4
"""2010-01-03""","""2010-01-04""","""A1""","""B1""","""S2""","""P2""",0.2
"""2010-01-07""","""2010-01-09""","""A1""","""B1""","""S6""","""P6""",0.6
"""2010-01-01""","""2010-01-05""","""A2""","""B2""","""S3""","""P5""",0.8
"""2010-01-02""","""2010-01-05""","""A2""","""B2""","""S8""","""P8""",0.8
"""2010-01-08""","""2010-01-10""","""A2""","""B3""","""S7""","""P7""",0.7


In [29]:
compare_start_prev_end = df8.start > df8.end.shift(periods=1)

compare_start_prev_end

shape: (7,)
Series: '' [bool]
[
	null
	false
	false
	true
	false
	false
	true
]

In [30]:
compare_start_prev_end.fill_none("zero")

shape: (7,)
Series: '' [bool]
[
	false
	false
	false
	true
	false
	false
	true
]

In [31]:
for g in df6.sort(["from", "to", "start", "end"]).groupby(["from", "to"]):
    print(g)

shape: (3, 7)
╭──────────────┬──────────────┬──────┬──────┬────────┬──────────┬──────╮
│ start        ┆ end          ┆ from ┆ to   ┆ source ┆ provider ┆ cost │
│ ---          ┆ ---          ┆ ---  ┆ ---  ┆ ---    ┆ ---      ┆ ---  │
│ str          ┆ str          ┆ str  ┆ str  ┆ str    ┆ str      ┆ f64  │
╞══════════════╪══════════════╪══════╪══════╪════════╪══════════╪══════╡
│ "2010-01-01" ┆ "2010-01-05" ┆ "A2" ┆ "B2" ┆ "S3"   ┆ "P3"     ┆ 0.3  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ "2010-01-01" ┆ "2010-01-05" ┆ "A2" ┆ "B2" ┆ "S5"   ┆ "P5"     ┆ 0.5  │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌┤
│ "2010-01-02" ┆ "2010-01-05" ┆ "A2" ┆ "B2" ┆ "S8"   ┆ "P8"     ┆ 0.8  │
╰──────────────┴──────────────┴──────┴──────┴────────┴──────────┴──────╯
shape: (1, 7)
╭──────────────┬──────────────┬──────┬──────┬────────┬──────────┬──────╮
│ start        ┆ end          ┆ from ┆ to   ┆ source ┆ provider ┆ cost │
│ ---          ┆ ---   

In [32]:
df7 = pd.DataFrame(data=[
    ["2010-01-01", "2010-01-05", "A", "1"],
    ["2010-01-03", "2010-01-07", "A", "2"],
    ["2010-01-04", "2010-01-06", "A", "3"],
], columns=["start", "end", "from", "to"]
)

df7.start = pd.to_datetime(df7.start)
df7.end = pd.to_datetime(df7.end)

df7

,start,end,from,to
0,2010-01-01,2010-01-05,A,1
1,2010-01-03,2010-01-07,A,2
2,2010-01-04,2010-01-06,A,3


In [33]:
df8 = df7.melt(id_vars=["from", "to"], var_name="__tmp_var", value_name="start").drop("__tmp_var", axis=1)

df8

,from,to,start
0,A,1,2010-01-01
1,A,2,2010-01-03
2,A,3,2010-01-04
3,A,1,2010-01-05
4,A,2,2010-01-07
5,A,3,2010-01-06


In [34]:
df9 = df8.sort_values(["from", "to"]).set_index("start")

df9

,from,to
start,,
2010-01-01,A,1
2010-01-05,A,1
2010-01-03,A,2
2010-01-07,A,2
2010-01-04,A,3
2010-01-06,A,3


In [35]:
groups = df9.groupby(["from", "to"])

In [36]:
# Drop the last date as we will create end = start + 1 for each date
# Feel free to set end = end - 1 before melting to avoid dropping the last row here
resampled_groups = [group_df.resample("D").ffill().iloc[:-1] for (_, group_df) in groups]

resampled_groups

[           from to
 start             
 2010-01-01    A  1
 2010-01-02    A  1
 2010-01-03    A  1
 2010-01-04    A  1,            from to
 start             
 2010-01-03    A  2
 2010-01-04    A  2
 2010-01-05    A  2
 2010-01-06    A  2,            from to
 start             
 2010-01-04    A  3
 2010-01-05    A  3]

In [37]:
df10 = pd.concat(resampled_groups)

df10

,from,to
start,,
2010-01-01,A,1
2010-01-02,A,1
2010-01-03,A,1
2010-01-04,A,1
2010-01-03,A,2
2010-01-04,A,2
2010-01-05,A,2
2010-01-06,A,2
2010-01-04,A,3


In [38]:
df11 = df10.groupby(["start", "from"]).agg({"to": ", ".join}).reset_index()

df11

,start,from,to
0,2010-01-01,A,1
1,2010-01-02,A,1
2,2010-01-03,A,"1, 2"
3,2010-01-04,A,"1, 2, 3"
4,2010-01-05,A,"2, 3"
5,2010-01-06,A,2


In [39]:
df11["end"] = df11["start"] + pd.Timedelta(days=1)

df11

,start,from,to,end
0,2010-01-01,A,1,2010-01-02
1,2010-01-02,A,1,2010-01-03
2,2010-01-03,A,"1, 2",2010-01-04
3,2010-01-04,A,"1, 2, 3",2010-01-05
4,2010-01-05,A,"2, 3",2010-01-06
5,2010-01-06,A,2,2010-01-07


In [40]:
df12 = merge_intervals_based_on_cols(df11, groupby_cols=["from", "to"], start_col=start_col, end_col=end_col)

df12

,start,end,from,to
0,2010-01-01,2010-01-03,A,1
1,2010-01-03,2010-01-04,A,"1, 2"
2,2010-01-04,2010-01-05,A,"1, 2, 3"
3,2010-01-05,2010-01-06,A,"2, 3"
4,2010-01-06,2010-01-07,A,2


In [41]:
def merge_intervals_and_combine_targets(
    df: pd.DataFrame,
    from_col: str = "from",
    to_col: str = "to",
    start_col: str = "start",
    end_col: str = "end",
) -> pd.DataFrame:
    df = df.melt(id_vars=[from_col, to_col], var_name="__tmp_var", value_name=start_col)
    df.drop("__tmp_var", axis=1, inplace=True)
    df.sort_values([from_col, to_col], inplace=True)
    df.set_index(start_col, inplace=True)
    
    groups = df.groupby([from_col, to_col])
    # Drop the last date as we will create end = start + 1 for each date
    # Feel free to set end = end - 1 before melting to avoid dropping the last row here
    resampled_groups = [group_df.resample("D").ffill().iloc[:-1] for (_, group_df) in groups]

    df = pd.concat(resampled_groups)
    df = df.groupby([start_col, from_col]).agg({to_col: ", ".join})
    df.reset_index(inplace=True)
    df[end_col] = df11[start_col] + pd.Timedelta(days=1)

    result = merge_intervals_based_on_cols(
        df,
        groupby_cols=[from_col, to_col],
        start_col=start_col,
        end_col=end_col
    )
    
    return result

In [42]:
merge_intervals_and_combine_targets(df7)

,start,end,from,to
0,2010-01-01,2010-01-03,A,1
1,2010-01-03,2010-01-04,A,"1, 2"
2,2010-01-04,2010-01-05,A,"1, 2, 3"
3,2010-01-05,2010-01-06,A,"2, 3"
4,2010-01-06,2010-01-07,A,2
